In [ ]:
from pathlib import Path

current_path = Path.cwd()


In [ ]:
def find_git_root(path: Path) -> Path:
    if (path / ".git").exists():
        return path
    if path == path.parent:
        raise FileNotFoundError("Git root not found")
    return find_git_root(path.parent)

In [ ]:
git_path = find_git_root(current_path)

In [ ]:
git_path

In [ ]:
from git import Repo

repo = Repo(git_path)

In [ ]:
from typing import Optional

def get_git_change(repo: Repo) -> Optional[str]:
    
    staged_change = repo.git.diff("--staged")
    
    if staged_change:
        print("Staged changes found")
        print(staged_change)
        return staged_change
    else:
        unstaged_change = repo.git.diff()
        if unstaged_change:
            print("Unstaged changes found")
            print(unstaged_change)
            return unstaged_change
        else:
            print("No changes found")
            return None
    
diff = get_git_change(repo)

diff

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

system_template = """
You are an AI assistant tasked with generating commit messages that strictly adhere to the commitizen conventions. Please follow these instructions to create commit messages:

1. Select the type of commit from the following options:
   - feat: A new feature
   - fix: A bug fix
   - docs: Documentation only changes
   - style: Changes that do not affect the meaning of the code (white-space, formatting, missing semi-colons, etc)
   - refactor: A code change that neither fixes a bug nor adds a feature
   - perf: A code change that improves performance
   - test: Adding missing tests or correcting existing tests
   - chore: Changes to the build process or auxiliary tools and libraries such as documentation generation

2. Specify the scope of the change (optional). This can help clarify which part of the codebase is affected.

3. Write a short, imperative mood description of the change (50 characters or less). This should be capitalized and not end with a period.

4. Provide a more detailed description of the changes if necessary. Wrap each line at 72 characters.

5. If there are any breaking changes, add a description with the prefix "BREAKING CHANGE:".

6. Add a footer with any relevant issue numbers or other references.

Based on this information, please generate a commit message that follows the commitizen format.

Example:
```
feat(auth): Implement two-factor authentication

- Add QR code generation for setup
- Implement time-based one-time password (TOTP) validation
- Update user model to store 2FA secret

BREAKING CHANGE: Users will need to set up 2FA on next login

Closes #123
```
"""


user_template = """
The git diff is below.

{diff}
"""

messages = [
    SystemMessage(system_template),
    HumanMessage(user_template)
]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

In [ ]:
result = prompt_template.invoke({"diff": diff})

result

In [ ]:
result.to_messages()

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
chain = prompt_template | model 

In [ ]:
result = chain.invoke({"diff": diff})

result.content

In [ ]:
user_template.format(diff="hello")